In [20]:
#importing libraries
import pandas as pd
import sqlite3
import matplotlib as plt
import seaborn as sns

In [22]:
df = pd.read_excel('/home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement.ods')
df.head(10)

,Receipt No.,Completion Time,Details,Transaction Status,Paid In,Withdrawn,Balance
0,REF8MUC88I,2023-05-15 20:19:22,Customer Transfer Fuliza MPesato - 07******212...,Completed,NaN,-50,0
1,REF8MUC88I,2023-05-15 20:19:22,OverDraft of Credit Party,Completed,50,NaN,50
2,REF9MU2IWF,2023-05-15 20:17:21,Merchant Payment Fuliza M-Pesato 7384394 - DAM...,Completed,NaN,-30,0
3,REF9MU2IWF,2023-05-15 20:17:21,OverDraft of Credit Party,Completed,30,NaN,30
4,REF7MT5G7N,2023-05-15 20:10:35,Customer Transfer Fuliza MPesato - 2547******5...,Completed,NaN,-10,0
5,REF7MT5G7N,2023-05-15 20:10:35,OverDraft of Credit Party,Completed,10,NaN,10
6,REF6MSH7KM,2023-05-15 20:05:47,Merchant Payment Fuliza M-Pesato 671957 - FISH...,Completed,NaN,-70,0
7,REF6MSH7KM,2023-05-15 20:05:47,OverDraft of Credit Party,Completed,70,NaN,70
8,REF5MQQC5T,2023-05-15 19:53:32,OD Loan Repayment to 232323 -M-PESA Overdraw,Completed,NaN,-150,0
9,REF0MQQ2C2,2023-05-15 19:53:32,Funds received from -07******440 diana mugwe,Completed,150,NaN,150


In [23]:
#checking if there are any null values in the dataset
df.isna().sum()

Receipt No.            346
Completion Time        347
Details                349
Transaction Status     348
Paid In               2550
Withdrawn             1781
Balance                348
dtype: int64

In [24]:
#checking the datatypes in the dataframe
df.dtypes

Receipt No.           object
Completion Time       object
Details               object
Transaction Status    object
Paid In               object
Withdrawn             object
Balance               object
dtype: object

In [25]:
#showing the total received amount
df['Paid In'] = pd.to_numeric(df['Paid In'], errors='coerce')

# Calculate the sum of the 'Paid In' column
total_received = df['Paid In'].sum()

print('{:,.2f}'.format(total_received))

468,474.82


In [26]:
#showing the total amount spent
df['Withdrawn'] = pd.to_numeric(df['Withdrawn'], errors='coerce')

#calculatiing the sum of the withdrawn column
total_spent = df['Withdrawn'].sum()

print('{:,.2f}'.format(abs(total_spent)))

468,381.71


In [39]:
import numpy as np
df = df.replace('', np.nan).dropna(thresh=1)

In [42]:
df.to_excel('output.ods', engine='odf')

In [43]:
df2 = pd.read_excel('/home/muregz/data_analysis/mpesa_statement_analysis/output.ods')

In [48]:
# Identify rows with non-parsable "Completion Time" values
df2 = df2[df2['Completion Time'].apply(lambda x: not pd.to_datetime(x, errors='coerce', format='%Y-%m-%d %H:%M:%S'))]



In [49]:
#converting the completed time column to datetime
df2['Completion Time'] = pd.to_datetime(df2['Completion Time'])

In [50]:
df2.dtypes

Unnamed: 0                     int64
Receipt No.                   object
Completion Time       datetime64[ns]
Details                       object
Transaction Status            object
Paid In                      float64
Withdrawn                    float64
Balance                       object
dtype: object